# Psychophysics Tutorial II: Threshold Estimation

Written by G.M. Boynton for Psych 448 at the University of Washington

Adapted for CSHL in June 2010, and revised for 2012 and 2014.

Translated to Python by Michael Waskom in 2018.

The goal of this tutorial is to define the detection threshold. We'll
simulate psychophysical data based on signal detection theory.  Then
we'll fit the simulated data with a parametric psychometric function
using a maximum likelihood procedure. Then we'll then pick off the
stimulus intensity level that predicts 80% correct performance.

It is suggested you look at the Psychophysics I tutorial first, which
covers signal detection theory, ROC curves and the 2AFC paradigm.


In [ ]:
%matplotlib inline
import numpy as np
from numpy import sqrt, log, exp
from scipy import stats
from scipy.stats import norm
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact
from IPython.display import display

## The psychometric function

Real psychophysicists don't report proportion correct as a dependent
measure.  They often don't report d-prime either.  Instead, they report
the 'threshold', which is the stimulus intensity required to produce a
desired level of performance.  A big advantage to the threshold is that
it is in the physical units of the stimulus, which can be more easily
compared across paradigms and labs.  Another advantage is that it can
produce a family of stimuli that all presumably produce the same internal
response.

There are various ways to find the intensity that yields a criterion
performance level.  We'll implement the a 3-down 1-up staircase method.
In this tutorial we'll simulate a subject's performance by assuming a
specific psychometric function that predicts performance in a 2AFC task.
This function will be a Weibull function.

## A SDT model to generate psychometric functions.

If you've gone through the Psycho_Tutorial_I tutorial you've learned that
for a 2AFC experiment under signal detection theory (SDT), d-prime can
be calculated from percent correct by:


intensityList = linspace(0,4,101);
dPrime = intensityList;
PC = normcdf(dPrime,0,sqrt(2));
figure(1)
clf
plot(intensityList,PC,'b-');
ylabel('Proportion correct');
xlabel('Stimulus intensity (d-prime)');
title('This is a cumulative normal');


In [ ]:
p_c = .8
d_prime = sqrt(2) * norm.ppf(p_c)

# We could also go the other way
assert norm.cdf(d_prime / sqrt(2)) == p_c

This means that if the physical stimulus strength is equivalent to
d-prime, then the expected psychometric function should be a cumulative
normal:

In [ ]:
d_prime = x = np.linspace(0, 4, 101)
p_c = norm.cdf(d_prime, 0, sqrt(2))

f, ax = plt.subplots(figsize=(5, 5))
ax.plot(x, p_c)
ax.set(xlabel="Stimulus intensity or $d'$",
       ylabel="Proportion correct",
       title="Cumulative normal")
f.tight_layout()

But if d-prime represents the signal-to-noise ratio for the internal
response, it does not typically increase linearly with physical stimulus
strength.  In most cases, like for luminance or contrast, it seems that
d-prime increases in a compressive fashion with stimulus strength.  A
simple compressive function is a power function with an exponent less
than 1.

If d-prime increases as a power function then then percent correct will
no longer be a cumulative normal:

In [ ]:
x = np.linspace(0, 30, 101)
d_prime = x ** .5
p_c = norm.cdf(d_prime / sqrt(2))
f, ax = plt.subplots(figsize=(5, 5))
ax.plot(x, p_c)
ax.set(xlabel="Stimulus intensity or $d'$",
       ylabel="Proportion correct",
       title="Not a cumulative normal")
f.tight_layout()

But we can plot on a log axis:

In [ ]:
f, ax = plt.subplots(figsize=(5, 5))
ax.plot(x[1:], p_c[1:])
ax.set(xscale="log",
       xlabel="Stimulus intensity or $d'$",
       ylabel="Proportion correct",
       title="This looks like a Weibull function")
f.tight_layout()

---

## The Weibull function

The Weibull cumulative distribution function is one of a variety of
functions that predict a psychometric function.  It has the general form:

$$y = 1 - e^{-(x/\lambda)^k},$$

where $x$ is the stimulus intensity and $y$ is the proportion correct.
$\lambda$ and $k$ are free parameters.  A reparameterized version for our purposes
is:

$$y = 1 - (1-g)\,e^{-(kx/t)^b},$$

where

$$k = -\log \bigg(\frac{1-a}{1-g}\bigg)^{1/b}.$$

With this parameterization, $g$ is the performance expected at chance
(0.5 in our case for 2AFC), $t$ is the threshold, and $a$ is the performance level
that defines the threshold.  That is, when $x=t$ then $y=a$.  The parameter $b$
determines the slope of the function.

Here's our version of our function:



In [ ]:
def weibull(x, b, t, g=.5):
    x = np.asarray(x)
    a = .5 ** (1 / 3)
    k = (-log((1 - a) / (1 - g))) ** (1 / b)
    y = 1 - (1 - g) * exp(-(k * x / t) ** b)
    return y

Let's define an interactive widget to let you play with the slope and threshold (actually the exponents) and see their effect:

In [ ]:
@interact
def weibull_tutorial(b_exp=(-4, 4, .5), t_exp=(-4, 4, .5)):
    x = np.logspace(-3, 3, 201, base=4)
    y = weibull(x, 4 ** b_exp, 4 ** t_exp)
    f, ax = plt.subplots(figsize=(5, 5))
    ax.plot(x, y, lw=2)
    ax.set_xscale("log", basex=4)
    ax.set(
        ylim=(.5, 1),
        xlabel="Intensity",
        ylabel="P(correct)",
    )
    f.tight_layout()

By varying the two parameters, we can generate a whole family of curves
that are good at describing psychometric functions in a 2AFC experiment.

In [ ]:
f, (ax_b, ax_t) = plt.subplots(1, 2, figsize=(7, 4),
                               sharex=True, sharey=True)

xs = ts = .0625, .25, 1, 4, 16, 64
bs = .5, 1, 2, 4
x = np.logspace(-2.5, 3.5, 101, base=4)

for b in bs:
    y = weibull(x, b=b, t=1)
    ax_b.plot(x, y, label=b)
    
for t in ts:
    y = weibull(x, b=2, t=t)
    ax_t.plot(x, y, label=t)

ax_b.legend(loc="lower right", title="$b$")
ax_t.legend(loc="lower right", title="$t$")
ax_b.set_xscale("log")
ax_b.set(
    xticks=xs,
    xticklabels=xs,
    xlabel="Intensity",
    ylabel="P(correct)",
)
f.tight_layout()

See how for each curve, when the intensity is equal to the threshold,
then the proportion correct is ~80%. So the variable $t$ determines
where on the x-axis the curve reaches 80%.


---

## The 3-down-1-up staircase

The method of constant stimuli is simple but has the disadvantage that
many of the trials are either too hard or too easy. These trials
contribute relatively little to our estimate of the subject's threshold.

Instead, we'll adjust the stimulus on each trial based on previous
responses using a 3-down 1-up 'staircase' procedure. Whenever the subject
makes an incorrect response, the next trial has an easier (larger)
intensity.  If the subject gets three trials in a row correct, the next
trial gets harder.

Staircase procedures deserves a tutorial on its own.  A 3-down 1-up is
the simplest.  There are many others, with names like Quest and Pest.

First, we need to define the exponent for the compressive nonlinearity
that transforms stimulus strength to d-prime. 0.3 is a good value to
simulate the dimension of stimulus contrast.


In [ ]:
noise_mean = 0
sd = 1
p = .3
n_trials = 200
correct_in_a_row = 0

# Control the randomness of the simulation
random_state = np.random.RandomState(seed=100)

# Initialize the results
trials = np.arange(n_trials) + 1
intensities = np.zeros(n_trials)
responses = np.zeros(n_trials)

# Start with the strongest stimulus
x_idx = len(xs) - 1

for i in range(n_trials):

    intensities[i] = x_i = xs[x_idx]
    signal_mean = x_i ** p

    noise = norm.rvs(noise_mean, sd, random_state=random_state)
    signal = norm.rvs(signal_mean, sd, random_state=random_state)
    responses[i] = response = signal > noise

    if response:
        correct_in_a_row += 1
        if correct_in_a_row == 3:
            x_idx = max(0, x_idx - 1)
            correct_in_a_row = 0
    else:
        correct_in_a_row = 0
        x_idx = min(x_idx + 1, len(xs) - 1)

We can plot the staircase showing which trials were answered correctly (white) or incorrectly (red) and how the intensity level was adjusted by these responses:

In [ ]:
f, ax = plt.subplots(figsize=(8, 4))

ax.set_yscale("log", basey=4)
ax.step(trials, intensities, c=".5")
ax.scatter(trials, intensities, c=responses,
           cmap="Reds_r", vmin=-.2, edgecolor=".5", zorder=3)
ax.set(
    xlim=(0, n_trials + 1),
    xlabel="Trial",
    ylabel="Intensity",
)
f.tight_layout()

After finishing the staircase, we can compile the results to generate a psychometric function:

In [ ]:
pmf_x = np.unique(intensities)
pmf_y = [np.mean(responses[intensities == x_i]) for x_i in pmf_x]
pmf_e = [stats.sem(responses[intensities == x_i]) for x_i in pmf_x]

In [ ]:
f, ax_pmf = plt.subplots(figsize=(5, 5))
ax_pmf.errorbar(pmf_x, pmf_y, pmf_e,
                marker="o", ls="")
ax_pmf.set_xscale("log", basex=4)
ax_pmf.set(
    xlabel="Intensity",
    ylabel="P(correct)"
)
f.tight_layout()

We could also have used seaborn to calculate the aggregate statistics
and error bars for us:

In [ ]:
f, ax = plt.subplots(figsize=(5, 5))
sns.lineplot(x=intensities, y=responses, ci=68,
             err_style="bars", marker="o", lw=0)
ax.set_xscale("log", basex=4)
ax.set(
    xlabel="Intensity",
    ylabel="P(correct)"
)
f.tight_layout()

## Threshold

The goal is to measure the minimum stimulus intensity that is 'reliably'
detected. But what does this mean? We could mean a stimulus intensity
that corresponds to above-chance performance.  More commonly, we choose
fix performance level, say 80% correct and estimate the intensity that
produces this level of performance. This is important.  Beware of people
who get excited about the ability to reliably 'see' stimuli below
'threshold'!

With a 3-down 1-up staircase in the limit, the intensity level should
gravitate toward a value that has an equal probability of getting easier
and getting harder. Thus, the probability of getting 3 in a row correct
is equal to 1/2.  This probability is the cube root of 1/2:

In [ ]:
(1 / 2) ** (1 / 3)

So a 3-down 1-up staircase method will naturally adjust the intensity
level to show trials that lead to about 80% correct.  Cool?  I think so.

How do we estimate the stimulus intensity that produces 80% correct
performance? Looking at the graph, if we interpolate, we can guess that
it's about 0.08.  But linear interpolation is a bad idea.  First, it
throws out information from all data except for two intensity values, and
second it requires the measured psychometric function data to be
monotonic.

Instead, we'll get to the heart of this tutorial and fit a smooth curve
to the psychometric function data and use this best-fitting curve to pick
off the threshold.

### A first guess at the parameters

Let's choose some initial parameters for a slope and threshold. We can
choose the values that we used to generate our data:

In [ ]:
p_guess = dict(b=1, t=.25)
y_guess = weibull(x, **p_guess)
ax_pmf.plot(x, y_guess, c=".2")
display(ax_pmf.figure)

Is this a good fit? To answer this we need a measure of how well the
curve fits the data. A common measure for curve fitting is a
sums-of-squared error (SSE), which is the sum of the squared deviations
between the curves and the data points.  However, for proportion-correct
data like this, SSE is not appropriate because deviations along the
proportion-correct do not have equal weights.  A 10% deviation for
performance around 50% is less meaningful than a 10% deviation around
90%.  You can see this by looking at the error bars.  They're always
small near 100%.

## Likelihood

For proportion-correct data (or any data generated through a binary
process), the appropriate measure is 'likelihood'. Here's how it's
calculated;

For a given trial, $i$, at a given stimulus intensity, $x_i$, the Weibull
function predicts the probability that the subject will get the answer
correct. So if the subject did get respond correctly, then the
probability of that happening is:

$$p_i = W(x_i)$$

Where $x_i$ is the intensity of the stimulus at trial $i$, and $W(x_i)$ is the
Weibull function evaluated at that stimulus intensity.  The probability
of an incorrect answer is, of course:

$$q_i = 1-W(x_i)$$

Assuming that all trials in the staircase are independent, then for a
given Weibull function, the probability of observing the entire sequence
of subject responses is:

$$\prod p_i = \prod W(x_i)$$

for correct trials, and

$$\prod q_i = \prod 1 - W(x_i)$$

for incorrect trials.

Here's how to do it in Python with our data set. First we evaluate the
Weibull function for the stimulus intensities used in the staircase:

In [ ]:
y = weibull(intensities, **p_guess)

Then we calculate the likelihood of observing our data set given this
particular choice of parameters for the Weibull:

In [ ]:
likelihood = np.prod(np.where(responses, y, 1 - y))
likelihood

We want to choose values of p.t and p.b to make this number as large as
possible. This is a really small number (in fact, someimtes it only gets displayed as 0).
This is sort of surprising since we thought that our choice of parameters for the
Weibull was reasonably good. The reason for this small number is that the product
of a bunch of numbers less than 1 gets really small. So small that it gets into the
tolerance for numpy to represent small numbers. To avoid this, we can take the
logarithm of the equation above - this expands the range of small numbers so tha
we're not going to run into machine tolerance problems:

In [ ]:
with np.errstate(all="ignore"):
    log_likelihood = np.sum(np.log(np.where(responses, y, 1 - y)))
log_likelihood

Why is this `nan` (or sometimes `inf`)? Turns out that this calculation can fail if the values of y reach 1
because the log of `1 - y` can't be computed.  The way around this is to pull
the values of y away from zero and 1 like this:

In [ ]:
eps = 1e-5
y = np.clip(y, eps, 1 - eps)

This is called a 'correction for guessing' and it deals with the fact
that subjects will never truly perform at 100% for any stimulus intensity
because human subjects will always make non-perceptual errors, like motor
errors, or have lapses in attention.

In [ ]:
log_likelihood = np.sum(np.log(np.where(responses, y, 1 - y)))
log_likelihood

This is a more reasonable magnitude to deal with. It's negative because
the log of a number less than 1 is negative. Larger numbers (less
negative) still correspond to 'good' fits.

Let's calculate the log likelihood for a different set of Weibull
parameters.

Here's a function that does a very similar calculation as above, using a vector of the parameters, but reverses the sign to that better fits are associated with smaller positive numbers. We do this because we will use optimization search algorithms that minimize the result of a function.

In [ ]:
def weibull_cost(p, intensities, responses, eps=1e-5):
    b, t = p
    y = np.clip(weibull(intensities, b, t), eps, 1 - eps)
    return -1 * np.sum(np.log(np.where(responses, y, 1 - y)), axis=0)

If we use the same parameters as before, we should get exactly the negative of the previous calculation:

In [ ]:
weibull_cost([p_guess["b"], p_guess["t"]], intensities, responses)

Let's try a different fit:

In [ ]:
p_better = dict(b=1, t=1)
weibull_cost([p_better["b"], p_better["t"]], intensities, responses)

We can visualize this by plotting this new prediction in red on the old graph:

In [ ]:
ax_pmf.plot(x, weibull(x, 1, 1), c="r")
display(ax_pmf.figure)

## The log-likelihood surface

How do we find the parameters that maximize the log likelihood?  `scipy`
has a function that does this in a sophisticated way.  But for fun, let's
just look at what the log likelihood is for a range of parameter values.
Since there are two parameters for the Weibull function, we can think of
the log likelihood as being a 'surface' with `b` and `y` being the x and
y axes.

In [ ]:
bs = np.linspace(.05, 2, 201)
ts = np.logspace(-3, 3, 202, base=4)

bb, tt = np.meshgrid(bs, ts, indexing="ij")
pp = bb.ravel(), tt.ravel()
ll = weibull_cost(pp, intensities[:, np.newaxis], responses[:, np.newaxis])
loglike_surf= ll.reshape(bb.shape)

In [ ]:
f, ax_ll = plt.subplots(figsize=(6, 5))
ax_ll.set_xscale("log", basex=4)
ax_ll.set(xlabel="t", ylabel="b")
cset = ax_ll.contour(tt, bb, loglike_surf, 50)
f.colorbar(cset)
f.tight_layout()

We can plot the most recent set of parameters as a symbol on this contour plot.

In [ ]:
ax_ll.scatter(
    [p_guess["t"], p_better["t"]],
    [p_guess["b"], p_better["b"]],
    c=[".2", "r"],
     )
ax_ll.figure

This figure is like a topographical map. Each line represents the values
of b and t that produce an equal value for log likelihood.

The figure shows a series of concentric shapes with the smallest circling
around `t=0.12` and `b = 1.6`.  This is the 'bottom' of our surface and is
close to the best choice of Weibull parameters.

## scipy's 'minimize' routine

Searching through the entire grid of possible parameters is clearly an
inefficient strategy (especially if there are even more parameters to
deal with).  Fortunately there is a whole science behind finding the best
parameters to minimize a function, and `scipy` has incorporated some of
the best in their function called 'minimize'.

TODO: The original tutorial includes a wrapper function that gives a slightly
nicer interface to the optimization. This would not be too hard to mirror,
but I'm punting for now.

To use the `minimize` function, we need to provide an objective function
(a function that takes parameters and returns the score we want to minimize),
an initial guess for the parameters (`x0`), and other arguments that should
be passed to the objective function each time it's called (`args`). The
`minimize` function is actually an interface to a large number of optimizers,
so we also need to pick the one we need to use:

In [ ]:
res = minimize(weibull_cost, x0=[1, .25], args=(intensities, responses), method="nelder-mead")
p_best = dict(zip(["b", "t"], res.x))
log_likelihood_best = res.fun

In [ ]:
print("Best parameters: b={b:.2f}, t={t:.2f}".format(**p_best))
print(f"Best score {log_likelihood_best:.2f}")

Note that the parameters changed from the initial values, and the
resulting log likelihood is better than the two used earlier (using
`p_guess` and `p_better`.  Let's plot the best-fitting parameters on the
contour plot in blue:

In [ ]:
ax_ll.scatter(p_best["t"], p_best["b"], c="b")
display(ax_ll.figure)

See how this best-fitting pair of parameters falls in the middle of the
circular-shaped contour.  This is the lowest point on the surface.

We now have the best fitting parameters.  Let's draw the best predictions
in blue on top of the psychometric function data:

In [ ]:
ax_pmf.plot(x, weibull(x, **p_best), c="C0")
display(ax_pmf.figure)

In [ ]:
t_c = (1 / 2) ** (1 / 3)
thresh_x = [ax_pmf.get_xlim()[0], p_best["t"], p_best["t"]]
thresh_y = [t_c, t_c, ax_pmf.get_ylim()[0]]
ax_pmf.plot(thresh_x, thresh_y, ls=":", c=".6", scalex=False, scaley=False)
display(ax_pmf.figure)

TODO: Fixed and free parameters

The MATLAB tutorial shows how to hold one of the parameters fixed using the custom interface.
Doing so without the interface is kind of a drag, so I'm going to skip. I'll revisit if
I decide to implement a similar interface function in Python.

## Increment thresholds

So far we've only talked about detecting a signal against noise.  It's
easy to use SDT to predict the ability to discriminate two stimuli that
differ in strength.  An example of this is the increment threshold, which
is the smallest increment in response that leads to some criterion
performance.

We can calculate the increment threshold function from the nonlinear
transducer function using our SDT model.  Recall that in a 2AFC task, the
threshold is the stimulus intensity (or increment) that produces a
criterion level of performance.  In our definition, the level of
performance is $(1/2)^(1/3) \approx .7937$ proportion correct.  Ths corresponds
to a d-prime value of:

In [ ]:
p = .3
d_prime = sqrt(2) * norm.ppf((1 / 2) ** (1 / 3))
print(f"d' = {d_prime:.2f}")

Suppose we have the following baseline intensity values:

In [ ]:
base_intensities = np.arange(11, dtype=np.float)

We want to estimate the increments in stimulus intensity that produce a
d-prime increment in internal response.  We'll find that increment using
a binary search algorithm in which we refine our estimates within
narrowing brackets of high and low estimates.

In [ ]:
# Starting intensities that are too low
lo = np.zeros_like(base_intensities)

# Starting intensities that are too high
hi = np.full_like(base_intensities, 40)

base_response = base_intensities ** p

n_iter = 20
for _ in range(n_iter):
    mid = (hi + lo) / 2
    increment_response = (base_intensities + mid) ** p - base_response
    too_high = increment_response > d_prime
    hi[too_high] = mid[too_high]
    lo[~too_high] = mid[~too_high]

pred_increment_thresh = (hi + lo) / 2

In [ ]:
f, (ax_r, ax_t) = plt.subplots(1, 2, figsize=(7, 5))
ax_r.plot(base_intensities, base_response, "o-")
ax_t.plot(base_intensities, pred_increment_thresh, "o-")
ax_r.set(
    title="Nonlinear transducer function",
    xlabel="Baseline intensity",
    ylabel="Internal response",
    ylim=(0, 5),
)
ax_t.set(
    title="Increment response function",
    xlabel="Baseline intensity",
    ylabel="Increment threshold",
    ylim=(0, 40),
)
f.tight_layout()

This is Weber's law!  Increment thresholds increase with baseline
contrast because of the compressive transducer function (power function)
between stimulus intensity and internal response.  As the baseline
intensity increases, the slope of the transducer function gets shallower
so you need a larger and larger stimulus increment to get the signal mean
to increase enough to obtain the d-prime needed for the threshold.

Mess with the exponent `p` to see the relationship between the two
curves.

Let's simulate the increment thresholds for the same range of baseline
stimulus strengths with our power-function model.  This will involve
repeated 3-down 1-up staircases for varying baseline intensities.  

In [ ]:
intensity_levels = .0156, .0625, .25, 1, 4, 16, 32, 64, 128
increment_thresholds = np.zeros_like(base_intensities)
n_trials = 1000
sd = 1

# Control the randomness of the simulation
random_state = np.random.RandomState(seed=100)

for j, base in enumerate(base_intensities):

    noise_mean = base ** p

    # Set up the staircase for this baseline
    idx = len(intensity_levels) - 1
    correct_in_a_row = 0
    intensities = np.zeros(n_trials)
    responses = np.zeros(n_trials)

    for i in range(n_trials):

        intensity = intensities[i] = intensity_levels[idx]

        signal_mean = (base + intensity) ** p
        signal = norm.rvs(signal_mean, sd, random_state=random_state)
        noise = norm.rvs(noise_mean, sd, random_state=random_state)

        response = responses[i] = signal > noise

        if response:
            correct_in_a_row += 1
            if correct_in_a_row == 3:
                idx = max(0, idx - 1)
                correct_in_a_row = 0
        else:
            correct_in_a_row = 0
            idx = min(idx + 1, len(intensity_levels) - 1)        

    res = minimize(weibull_cost, x0=[1, .25], args=(intensities, responses), method="nelder-mead")
    increment_thresholds[j] = res.x[1]

ax_t.plot(base_intensities, increment_thresholds, "o",
          ms=10, mew=2, mfc="none")
display(ax_t.figure)

There should be a reasonable match between our simulated data and the
expected increment response function.  The fit should improve with
increasing number of trials per staircase.

What good is this? The goal of the psychophysicist is to take behavioral
results (psycho) to learn something about the transformation between the
physical stimulus (physics) and the internal response.  This simulation
shows how you can predict in increment response function for a given
nonlinear transducer function.  But we can go the other way too.  For a
given psychophysical increment response function we can find the best
nonlinear transducer function that predicts our data.  

It turns out that for contrast increments, the psychophysical contrast
increment threshold function (sometimes called threshold vs. contrast or
TvC function) is surprisingly consistent with contrast response functions
measured in V1 with fMRI.  Moreover, the effects of surround stimuli and
attention affect psychophysical and fMRI responses in  a consistent way.

----

## Exercises

1. Find your own stimulus intensity threshold by fitting the Weibull
function to your own psychophysical data generated in Lesson 4.

2. Run 50 trials with a stimulus intensity level set at your own
threshold and see how close the proportion correct is to 80%

3. What happens when you use different initial parameters when calling
`minimize`?  Try some wild and crazy starting values.  You might find some
interesting 'fits'.  This illustrates the need to choose sensible initial
values.  If your function surface is convoluted then a bad set of initial
parameters may settle into a 'local minimum'.

4. How would you obtain an estimate of the reliability of the threshold
measure?  After all, $t$ is just a statistic based on the values in
'results'.

5. Run the increment threshold simulation for an exponent of 1.  What
happens and why?  